# How to run JANUS and JANUS + C

In [ ]:
!pip install janus-ga

## Define fitness function from tartarus

In [ ]:
import sys, os
sys.path.append('..')

from tartarus import pce

def fitness_function(smi: str):
    dipole, hl_gap, lumo, obj, pce_1, pce_2, sas = pce.get_properties(smi)
    return pce_1 - sas

## Get imports and set SELFIES constraints

In [ ]:
from janus import JANUS
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, RDConfig, Descriptors
RDLogger.DisableLog("rdApp.*")

import pandas as pd
import selfies

default_constraints = selfies.get_semantic_constraints()
new_constraints = default_constraints
new_constraints['S'] = 2
new_constraints['P'] = 3
selfies.set_semantic_constraints(new_constraints)  # update constraints

## Get the dataset for running

In [ ]:
data_path       = '../datasets/hce.csv'
df = pd.read_csv(data_path)

# creating starting_smiles.txt
with open('starting_smiles.txt', 'w') as f:
    for i, r in df.iterrows():
        f.write(r['smiles']+'\n')

## Start the JANUS agent and run GA

To use JANUS+C, set `use_classifier = True` in `params_dict`.

In [ ]:
def custom_filter(smi: str):
    """ Function that takes in a smile and returns a boolean.
    True indicates the smiles PASSES the filter.
    """
    # smiles length filter
    if len(smi) > 81 or len(smi) == 0:
        return False
    else:
        return True

# all parameters to be set, below are defaults
params_dict = {
    # Number of iterations that JANUS runs for
    "generations": 10,

    # The number of molecules for which fitness calculations are done, 
    # exploration and exploitation each have their own population
    "generation_size": 250,
    
    # Number of molecules that are exchanged between the exploration and exploitation
    "num_exchanges": 5,

    # Callable filtering function (None defaults to no filtering)
    "custom_filter": custom_filter,

    # Fragments from starting population used to extend alphabet for mutations
    "use_fragments": True,

    # An option to use a classifier as selection bias
    "use_classifier": False,
}

agent = JANUS(
    work_dir = 'RESULTS',
    fitness_function = fitness_function,
    start_population = 'starting_smiles.txt'
    **params_dict
)

agent.run()
